In [1]:
from typing import NamedTuple, Tuple, Dict, List, Union
import sys
import math
import torch
import random
import pandas as pd
from tqdm import tqdm


DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

<center><img src="media/rnn-transformer-meme.png" style="width: 500px;"/></center>

## Что же такое этот Transformer?
Transformer - модель полностью основанная только на Attention механизме.
<table>
    <td> <img src="media/transformer-architecture.png" style="width: 500px;"/> </td>
    <td> <img src="media/dot-product-multi-head.png" style="width: 600px;"/> </td>
</table>

В качестве Norm используют [LayerNorm by Hinton](https://arxiv.org/abs/1607.06450), так как он значительно лучше работает для NLP задач и не зависит от размера Batch, благодаря чему можно делать онлайн-обучение. Также для LayerNorm нет необходимости делать скользящее средней по expectation и variance.
Теперь порисуем где-нибудь модель и напишем псевдокод для каждой части.

## Преимущества Transformer
<center><img src="media/lstm-schmidhuber-meme.jpg" style="width: 500px;"/></center>

1. Быстрее.
2. Проще распаралелить.
3. Лучше работает с длинными последовательностями (так как расстояние от любого другого токена равно `O(1)`, в то время как у RNN `O(n)`).
4. Интерпретируемость.

## Напишем свой MultiHead Self-Attention

In [2]:
class MultiHeadAttentionOutput(NamedTuple):
    """Result of MultiHeadAttention Module call."""

    values: torch.Tensor
    attention: torch.Tensor


class MultiHeadAttention(torch.nn.Module):
    """
    Compute Multi-Head Attention like in "Attention Is All You Need" paper.
    For simplicity assume that value dimension equals query and key dimension.

    Parameters
    ----------
    num_heads : `int`, required
        Number of heads for Self-Attention.
    hidden_size : `int`, required
        Hidden size for projection in Self-Attention.
    bias : `bool`, optional (default = `True`)
        Whether to include bias for projection or not.
    dropout : `float`, optional (default = `0.1`)
        Dropout probability for Self-Attention after softmax.
    output_size : `int`, optional (default = `None`)
        Size for output projection. If None hidden size is used.
    attention_fill_value : `float`, optional (default = `1e-32`)
        Fill value for attention before softmax if mask is passed if forward.
    """

    def __init__(
        self,
        num_heads: int,
        hidden_size: int,
        bias: bool = True,
        dropout: float = 0.1,
        output_size: int = None,
        attention_fill_value: float = 1e-32
    ) -> None:
        super().__init__()
        assert hidden_size % num_heads == 0
        self.attn_size = hidden_size // num_heads
        self.num_heads = num_heads
        self.projections = torch.nn.ModuleDict({
            "query": torch.nn.Linear(hidden_size, hidden_size, bias=bias),
            "key": torch.nn.Linear(hidden_size, hidden_size, bias=bias),
            "value": torch.nn.Linear(hidden_size, hidden_size, bias=bias)
        })
        self.output = torch.nn.Linear(hidden_size, output_size or hidden_size)
        self.attention_fill_value = attention_fill_value
        if dropout:
            self.dropout = torch.nn.Dropout(p=dropout)
        else:
            self.dropout = lambda x: x

    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: torch.Tensor = None
    ) -> MultiHeadAttentionOutput:
        # query ~ (batch size, seq length, hidden size)
        # key ~ (batch size, seq length, hidden size)
        # value ~ (batch size, seq length, hidden size)
        # mask ~ (batch size, seq length)
        # TODO: From here
        # Because we decided to keep the model simple
        # then query.size = key.size = value.size
        original_size = value.size()
        # 1) Linear projections in batch from
        # (batch size, seq length, hidden size) => (batch size, num heads, seq length, attn size).
        query = self.multi_head_rearrange(self.projections["query"](query))
        key = self.multi_head_rearrange(self.projections["key"](key))
        value = self.multi_head_rearrange(self.projections["value"](value))
        # 2) Apply self-attention.
        # output ~ (batch size, num heads, seq length, attn size)
        # attn ~ (batch size, num heads, seq length, seq length)
        output, attn = self.self_attention(query, key, value, mask=mask)
        # 3) Rearrange back to normal.
        # output ~ (batch size, seq length, hidden size)
        output = (
            output
            .transpose(1, 2).contiguous()
            .view(*original_size)
        )
        return MultiHeadAttentionOutput(self.output(output), attn)

    def self_attention(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # query ~ (batch size, num heads, seq length, attn size)
        # key ~ (batch size, num heads, seq length, attn size)
        # value ~ (batch size, num heads, seq length, attn size)
        # mask ~ (batch size, seq length)
        # query @ key ~ (batch size, num heads, seq length, seq length)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.attn_size)
        if mask is not None:
            # Add dimensions for masked_fill to work
            while mask.dim() < scores.dim():
                mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask.eq(0), self.attention_fill_value)
        probs = torch.softmax(scores, dim=-1)
        probs = self.dropout(probs)
        # probs @ value ~ (batch size, num heads, seq length, attn size)
        return torch.matmul(probs, value), probs

    def multi_head_rearrange(self, tensor: torch.Tensor) -> torch.Tensor:
        return (
            tensor
            .view(tensor.size(0), -1, self.num_heads, self.attn_size)
            .transpose(1, 2)
        )

Зафиксировать seed и посмотрим работает ли))

In [3]:
tensor = torch.randn(10, 15, 256)
module = MultiHeadAttention(num_heads=4, hidden_size=256, dropout=0.0, bias=False)
module(tensor, tensor, tensor).values.size()

torch.Size([10, 15, 256])

Ну а теперь проверим с PyTorch MultiheadAttention

In [4]:
from torch.nn.modules.activation import MultiheadAttention as MultiheadAttentionTorch
# Instantiate Module
torch_attention = MultiheadAttentionTorch(embed_dim=256, num_heads=4, dropout=0.0, bias=False)
# Change parameters
torch_attention._qkv_same_embed_dim = False
# Projections
torch_attention.q_proj_weight = module.projections["query"].weight
torch_attention.k_proj_weight = module.projections["key"].weight
torch_attention.v_proj_weight = module.projections["value"].weight
torch_attention.out_proj = module.output
# Permute tensor because PyTorch MultiheadAttention
# accepts (sequence length, batch size, embedding size)
torch_attn_tensor = tensor.permute(1, 0, 2).contiguous()
assert torch.allclose(
    torch_attention(torch_attn_tensor, torch_attn_tensor, torch_attn_tensor)[0].permute(1, 0, 2),
    module(tensor, tensor, tensor).values
)

Сделать модель на основе Transformer в PyTorch.

Загрузим данные для NMT French-English

In [11]:
! ls

_about.txt  data  fra.txt  sample_data


In [12]:
# Create data dir if not exists
!mkdir -p data
# Load dataset
!wget https://www.manythings.org/anki/fra-eng.zip -O data/fra-eng.zip
# Unarchive dataset
! unzip -q data/fra-eng.zip
# Rename for build_dataset to work
!mv fra.txt eng-fra.txt
!mv eng-fra.txt data/eng-fra.txt

--2021-12-06 05:02:35--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.186.54, 104.21.92.44, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.186.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6515217 (6.2M) [application/zip]
Saving to: ‘data/fra-eng.zip’

data/fra-eng.zip    100%[===================>]   6.21M  8.14MB/s    in 0.8s    

2021-12-06 05:02:36 (8.14 MB/s) - ‘data/fra-eng.zip’ saved [6515217/6515217]

replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Ниже код для предобработки данных. Не обращайте на него внимание, он взят из [туториала PyTorch](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

In [18]:
import re
import random
import unicodedata


SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def read_langs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalize_string(s) for s in l.rstrip().split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filter_pair(p):
    return (
        len(p[0].split(' ')) < MAX_LENGTH
        and len(p[1].split(' ')) < MAX_LENGTH
        and p[1].startswith(eng_prefixes)
    )


def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]


def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_langs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


def build_dataset():
    # Prepare data
    input_lang, output_lang, pairs = prepare_data('eng', 'fra', True)
    # Get random pairs
    random.seed(13)
    random.shuffle(pairs)
    train_size = int(len(pairs) * 0.8)
    train_pairs = pairs[:train_size]
    test_pairs = pairs[train_size:]
    # Save datasets
    with open('data/train.txt', 'w', encoding='utf-8') as file:
        file.write('\n'.join(['\t'.join(pair) for pair in train_pairs]))
    with open('data/test.txt', 'w', encoding='utf-8') as file:
        file.write('\n'.join(['\t'.join(pair) for pair in test_pairs]))

In [20]:
build_dataset()

Reading lines...
Read 191954 sentence pairs
Trimmed to 14083 sentence pairs
Counting words...
Counted words:
fra 5054
eng 3260


### Подготовим LabelEncoder и Dataset в стилье PyTorch

In [ ]:
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader


SOS = "<start>"
EOS = "<end>"
PAD = "<pad>"


class Vocabulary:
    """
    Vocabulry construct label encoding (token -> index)
    and labeld decoding (index -> token) based on datasets.
    """

    def __init__(self, **datasets) -> None:
        self._word2index = defaultdict(dict)
        self._index2word = defaultdict(dict)
        self._setup_indexers()
        for name, dataset in datasets.items():
            self._iterate_dataset(name, dataset)

    @property
    def word2index(self):
        return self._word2index

    @property
    def index2word(self):
        return self._index2word

    def get_size(self, field: str) -> int:
        return len(self._word2index[field])

    def _iterate_dataset(self, name: str, dataset: Dataset) -> None:
        for sample in tqdm(dataset, desc=f"Building vocab from {name}", total=len(dataset)):
            self._iterate_sample(sample)

    def _iterate_sample(self, sample: Dict[str, List[str]]) -> None:
        for field, sentence in sample.items():
            for word in filter(
                lambda x: x not in self._word2index[field], sentence
            ):
                index = len(self._word2index[field])
                self._word2index[field][word] = index
                self._index2word[field][index] = word

    def _setup_indexers(self) -> None:
        for token_type in [PAD, SOS, EOS]:
            index = len(self._word2index["source"])
            self._word2index["source"][token_type] = index
            self._word2index["target"][token_type] = index
            self._index2word["source"][index] = token_type
            self._index2word["target"][index] = token_type


class NMTDataset(Dataset):
    """
    PyTorch Dataset for NMT task.

    Parameters
    ----------
    data_path : `str`, required
        Path to dataset.
    """

    def __init__(self, data_path: str) -> None:
        self._dataset = []
        self._indexer = None
        with open(data_path, "r", encoding="utf-8") as file:
            for sample in tqdm(file, desc=f"Reading dataset at {data_path} in memory"):
                source, target = sample.rstrip().split('\t')
                self._dataset.append({
                    "source": source.split() + [EOS],
                    "target": [SOS] + target.split() + [EOS],
                })

    def __getitem__(self, idx: int) -> Dict[str, List[str]]:
        return {
            k: [self._indexer[k][x] if self._indexer else x for x in v]
            for k, v in self._dataset[idx].items()
        }

    def __len__(self) -> int:
        return len(self._dataset)

    def index_with(self, vocab: Vocabulary) -> None:
        self._indexer = vocab.word2index

In [ ]:
from torch.nn.utils.rnn import pad_sequence


# collate_fn в DataLoader вызывается чуть ли не в самом конце сборки одного батча
# поэтому это отличный способ сделать дополнительную предобработку данных
# в нашем случае нам необходимо сделать padding и сделать маску
class CollateBatch:
    """
    Collate Function for DataLoader as Class to perform postprocessing.
    We better use classes because it is better for code structure
    and much more convenient to build custom functions for batch of samples such pin_memory.

    Parameters
    ----------
    instances : `List[Dict[str, List[int]]]`
        List of samples as dicts from PyTorch Dataset.
    """

    def __init__(self, instances: List[Dict[str, List[int]]]) -> None:
        batch = self._form_batch(instances)
        # Pad sentences as each sentence is of different size
        self.source_tokens = pad_sequence([torch.Tensor(x) for x in batch['source']], batch_first=True).long()
        self.target_tokens = pad_sequence([torch.Tensor(x) for x in batch['target']], batch_first=True).long()
        # Construct mask to identify padding
        self.source_mask = self.source_tokens.ne(0)
        self.target_mask = self.target_tokens.ne(0)

    def pin_memory(self):
        """Pin memory for fast data transfer on CUDA."""
        self.__dict__ = {
            prop: tensor.pin_memory()
            for prop, tensor in self.__dict__.items()
        }
        return self

    def to_device(
        self,
        device: Union[str, torch.device],
        **extra_params
    ) -> Dict[str, torch.Tensor]:
        """Helper function to send batch to device and convert it to dict."""
        return {
            prop: tensor.to(device=device, **extra_params)
            for prop, tensor in self.__dict__.items()
        }

    @staticmethod
    def _form_batch(instances: List[Dict[str, List[int]]]) -> Dict[str, List[List]]:
        """Consturct normal batched data as dict from list of dicts."""
        tensor_dict = defaultdict(list)
        for instance in instances:
            for field, tensor in instance.items():
                tensor_dict[field].append(tensor)
        return tensor_dict

In [ ]:
# Building datasets
train_dataset = NMTDataset('./data/train.txt')
validation_dataset = NMTDataset('./data/test.txt')
# Building Vocabulry to convert words to indecies sequence
vocab = Vocabulary(
    train_dataset=train_dataset,
    validation_dataset=validation_dataset
)
# Index datasets with Vocabulary
train_dataset.index_with(vocab)
validation_dataset.index_with(vocab)
# Building DataLoaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=1,
    pin_memory=True,
    collate_fn=CollateBatch,
    shuffle=True,
)
validation_dataloader = DataLoader(
    dataset=validation_dataset,
    batch_size=32,
    num_workers=1,
    pin_memory=True,
    collate_fn=CollateBatch,
)

Reading dataset at ./data/train.txt in memory: 10661it [00:00, 278516.06it/s]
Reading dataset at ./data/test.txt in memory: 2666it [00:00, 294837.70it/s]
Building vocab from validation_dataset: 100%|██████████| 2666/2666 [00:00<00:00, 143073.01it/s]


### Напишем модель на основе LSTM для NMT задачи

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class LSTMEncoderOutput(NamedTuple):
    final_state: torch.Tensor
    cell_state: torch.Tensor


class LSTMEncoder(torch.nn.Module):
    """
    Decode text data with LSTM Module.

    Parameters
    ----------
    input_size : `int`, required
        Hidden size of input tokens.
    hidden_size : `int`, required
        Hidden size for LSTM.
    num_layers : `int`, optional (default = `1`)
        Number of stacked LSTM modules.
    dropout : `int`, optional (default = `0.2`)
        Dropout for LSTM module.
    """
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_layers: int = 1,
        dropout: float = 0.2
    ) -> None:
        super().__init__()
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._encoder = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=True,
        )

    def forward(self, source_tokens: torch.Tensor, mask: torch.Tensor) -> LSTMEncoderOutput:
        # source_tokens ~ (batch size, sequence length, hidden size)
        # mask ~ (batch size, sequence length)
        # 1) Pack sequence to skip processing of padding in tokens tensor
        # (LSTM returns 0 on padding)
        # Get lengths of each sequence
        lenghts = mask.long().sum(dim=-1)
        # Pack sequence to skip padding in RNN
        packed_sequence_input = pack_padded_sequence(
            input=source_tokens,
            lengths=lenghts.data.tolist(),
            batch_first=True,
            # Let pack_padded do the sorting for ourselves
            enforce_sorted=False,
        )
        # 2) Encode PackedSequence
        padded_encoded_sequence, (final_hidden_state, cell_state) = self._encoder(packed_sequence_input)
        # 3) Unpack encoded sequence
        # encoded_sequence ~ (batch size, sequence length, LSTM hidden size * 2)
        encoded_sequence = pad_packed_sequence(padded_encoded_sequence, batch_first=True)
        # Take average over frist dimension as it returns both forward and backward paths of LSTM
        return LSTMEncoderOutput(
            final_hidden_state.mean(0).unsqueeze(0),
            cell_state.mean(0).unsqueeze(0)
        )

    def get_input_size(self):
        return self._input_size

    def get_output_size(self):
        # Return bidirectional LSTM so mul by 2
        return self._hidden_size * 2


class LSTMDecoder(torch.nn.Module):
    """
    Decode sequence for NMT task with LSTM Module.

    Parameters
    ----------
    input_size : `int`, required
        Hidden size of input tokens.
    hidden_size : `int`, required
        Hidden size for LSTM.
    num_layers : `int`, optional (default = `1`)
        Number of stacked LSTM modules.
    dropout : `int`, optional (default = `0.2`)
        Dropout for LSTM module.
    """
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_layers: int = 1,
        dropout: float = 0.2
    ) -> None:
        super().__init__()
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._decoder = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
        )

    def forward(
        self,
        target_tokens: torch.Tensor,
        encoder_output: LSTMEncoderOutput,
        mask: torch.Tensor,
    ) -> torch.Tensor:
        # target_tokens ~ (batch size, sequence length, hidden size)
        # mask ~ (batch size, sequence length)
        # 1) Pack sequence to skip processing of padding in tokens tensor
        # (LSTM returns 0 on padding)
        lenghts = mask.long().sum(dim=-1)
        # Pack sequence to skip padding in RNN
        packed_sequence_input = pack_padded_sequence(
            input=target_tokens,
            lengths=lenghts.data.tolist(),
            batch_first=True,
            # Let pack_padded do the sorting for ourselves
            enforce_sorted=False,
        )
        # 2) Encode PackedSequence
        padded_decoded_sequence = self._decoder(packed_sequence_input, encoder_output)[0]
        # 3) Unpack encoded sequence
        # decoded_sequence ~ (batch size, sequence lenght, LSTM hidden size)
        decoded_sequence, _ = pad_packed_sequence(padded_decoded_sequence, batch_first=True)
        return decoded_sequence

    def get_input_size(self):
        return self._input_size

    def get_output_size(self):
        return self._hidden_size

### Напишем модель на основе Transformer для NMT задачи

In [ ]:
class TransformerEncoderOutput(NamedTuple):
    memory: torch.Tensor
    mask: torch.Tensor


class PositionalEncoding(torch.nn.Module):
    """
    Positional encoding for Transformer

    Parameters
    ----------
    hidden_size : `int`, required
        Hidden size of positional encoding.
        Must match hidden size of input tokens.
    dropout : `float`, required
        Dropout probability after positional encoding addition.
        If None dropout is not considered.
    max_len : `int`, optional (default = `5000`)
        Maximum sequence length to construct Positional Encoding. 
    """

    def __init__(self, hidden_size: int, dropout: float, max_len: int = 5000):
        super().__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, hidden_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, hidden_size, 2, dtype=torch.float) * -(math.log(10000.0) / hidden_size)
        )
        # First sin, then cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # Make torch.nn.Parameter from pe
        self.pe = torch.nn.Parameter(pe.unsqueeze(0), requires_grad=False)
        # Set dropout
        if dropout:
            self._dropout = nn.Dropout(p=dropout)
        else:
            self._dropout = lambda x: x

    def forward(self, tokens: torch.Tensor) -> torch.Tensor:
        tokens = tokens + self.pe[:, :tokens.size(1)]
        return self._dropout(tokens)


# PyTorch Transformer is somewhat weirdly implemented
# so you better write one yourself but for now let's use PyTorch Transformer
class TransformerEncoder(torch.nn.Module):
    """
    Encode input tokens with Transformer.

    Parameters
    ----------
    input_size : `int`, required
        Hidden size of input tokens.
    num_layers : `int`, required
        Number of stacked Transformers.
    feedforward_hidden_dim : `int`, optional (default = `2048`)
        Hidden size for feedforward layer in Transformer.
    num_attention_heads : `int`, optional (default = `8`)
        Number of attention heads in Transformer.
    dropout_prob : `float`, optional (default = `0.1`)
        Dropout probability for Transformer.
    activation : `str`, optional (default = `"relu"`)
        The activation function of intermediate layer, relu or gelu.
    """

    def __init__(
        self,
        input_size: int,
        num_layers: int,
        feedforward_hidden_dim: int = 2048,
        num_attention_heads: int = 8,
        dropout_prob: float = 0.1,
        activation: str = "relu",
    ) -> None:
        super().__init__()
        layer = torch.nn.TransformerEncoderLayer(
            d_model=input_size,
            nhead=num_attention_heads,
            dim_feedforward=feedforward_hidden_dim,
            dropout=dropout_prob,
            activation=activation,
        )
        self._transformer = torch.nn.TransformerEncoder(layer, num_layers)
        self._positional_encoding = PositionalEncoding(
            hidden_size=input_size,
            dropout=None,
        )
        self._input_size = input_size

    def forward(self, source_tokens: torch.Tensor, mask: torch.Tensor) -> TransformerEncoderOutput:
        # source_tokens ~ (batch size, sequence length, hidden size)
        # mask ~ (batch size, sequence length)
        tokens = self._positional_encoding(source_tokens)
        # For some reason the torch transformer expects the shape (sequence, batch, features), not the more
        # familiar (batch, sequence, features), so we have to fix it.
        # tokens ~ (sequence length, batch size, hidden size)
        tokens = tokens.permute(1, 0, 2)
        # For some other reason, the torch transformer takes the mask backwards.
        tokens_mask = ~mask
        # output ~ (sequence length, batch size, hidden size)
        output = self._transformer(tokens, src_key_padding_mask=tokens_mask)
        # output ~ (batch size, sequence length, hidden size)
        output = output.permute(1, 0, 2)
        return TransformerEncoderOutput(output, mask)

    def get_input_size(self) -> int:
        return self._input_size

    def get_output_size(self) -> int:
        return self._input_size


class TransformerDecoder(torch.nn.Module):
    """
    Decode sequence for NMT task with Transformer.

    Parameters
    ----------
    input_size : `int`, required
        Hidden size of input tokens.
    num_layers : `int`, required
        Number of stacked Transformers.
    feedforward_hidden_dim : `int`, optional (default = `2048`)
        Hidden size for feedforward layer in Transformer.
    num_attention_heads : `int`, optional (default = `8`)
        Number of attention heads in Transformer.
    dropout_prob : `float`, optional (default = `0.1`)
        Dropout probability for Transformer.
    activation : `str`, optional (default = `"relu"`)
        The activation function of intermediate layer, relu or gelu.
    """

    def __init__(
        self,
        input_size: int,
        num_layers: int,
        feedforward_hidden_dim: int = 2048,
        num_attention_heads: int = 8,
        dropout_prob: float = 0.1,
        activation: str = "relu",
    ) -> None:
        super().__init__()
        layer = torch.nn.TransformerDecoderLayer(
            d_model=input_size,
            nhead=num_attention_heads,
            dim_feedforward=feedforward_hidden_dim,
            dropout=dropout_prob,
            activation=activation
        )
        self._transformer = torch.nn.TransformerDecoder(layer, num_layers=num_layers)
        self._input_size = input_size

    def forward(
        self,
        target_tokens: torch.Tensor,
        encoder_output: TransformerEncoderOutput,
        mask: torch.Tensor
    ) -> torch.Tensor:
        # target_tokens ~ (batch size, sequence length, hidden size)
        # mask ~ (batch size, sequence length)
        # For some reason the torch transformer expects the shape (sequence, batch, features), not the more
        # familiar (batch, sequence, features), so we have to fix it.
        # target_tokens ~ (sequence length, batch size, hidden size)
        # memory ~ (sequence length, batch size, hidden size)
        target_tokens = target_tokens.permute(1, 0, 2)
        memory = encoder_output.memory.permute(1, 0, 2)
        # For some other reason, the torch transformer takes the mask backwards.
        target_mask = ~mask
        memory_mask = ~encoder_output.mask
        # output ~ (sequence length, batch size, hidden size)
        output = self._transformer(
            target_tokens, memory,
            tgt_key_padding_mask=target_mask,
            memory_key_padding_mask=memory_mask,
        )
        # output ~ (batch size, sequence length, hidden size)
        output = output.permute(1, 0, 2)
        return output

    def get_input_size(self) -> int:
        return self._input_size

    def get_output_size(self) -> int:
        return self._input_size

### Напишем Loss под нашу задачу

До этого мы использовали дефолтные лоссы в PyTorch, но теперь нам нужно написать свой лосс, так как мы хотим посчитать `CrossEntropyLoss` по тензору из 3 размерностей, в то время как `PyTorch CrossEntropyLoss` принимает тензоры размерности 2.

Возможно в примерах в интернете вы виделе как пишут свой лосс через `torch.autograd.Function`. Это хорошая практики и отличная тренировка писать ручками свой Backward, однако такой подход лучше всего подходит, когда у вас сложный лосс и там возникают for. Для нашего лосс это не нужно, поэтому сдалем всё через знакомый нам `torch.nn.Module`.

Очевидное решение, это просто сделать reshape и соединить `batch size` и `sequence length`, однако в таком случае мы будет считать лосс ещё и по `padding`, что нам совсем не нужно.

Поэтому напишем свой `CrossEntropyLoss`, который не будет учитывать `padding`. Его идея будет заключаться в том, чтобы взять логиты под индексами, которые не являются `padding`. Для этого очень хорошо подходит операция [torch.gather](https://pytorch.org/docs/stable/generated/torch.gather.html#torch-gather), которая буквально переводится как "собрать".

In [ ]:
class SequenceNLLLoss(torch.nn.Module):
    """NLL Loss for NMT task that ignores computing loss on paddding."""
    def __init__(self, size_average: bool = True) -> None:
        super().__init__()
        self._size_average = size_average

    def forward(
        self,
        logits: torch.Tensor,
        target: torch.Tensor,
        weights: torch.FloatTensor,
    ) -> torch.Tensor:
        # logits ~ (batch size, sequence length, num_classes)
        # target ~ (batch size, sequence length)
        # weights ~ (batch size, sequence length)
        non_batch_dims = tuple(range(1, weights.dim()))
        # weights_batch_sum  (batch size,)
        weights_batch_sum = weights.sum(dim=non_batch_dims)
        # logits_flat ~ (batch size * sequence length, num_classes)
        logits_flat = logits.view(-1, logits.size(-1))
        log_probs_flat = torch.log_softmax(logits_flat, dim=-1)
        # targets_flat ~ (batch size * sequence length, 1)
        targets_flat = target.view(-1, 1).long()
        # nll_loss ~ (batch size * sequence length,)
        # Gather numbers from log_probs_flat in targets_flat indices
        nll_loss = -torch.gather(log_probs_flat, dim=1, index=targets_flat)
        # nll_loss ~ (batch size, sequence length)
        nll_loss = nll_loss.view(*target.size()) * weights
        # per_batch_nll_loss ~ (batch size,)
        per_batch_nll_loss = nll_loss.sum(non_batch_dims) / torch.clamp(weights_batch_sum, min=1e-13)
        if self._size_average:
            num_non_empty_sequences = torch.clamp(weights_batch_sum.gt(0).float().sum(), min=1e-13)
            return per_batch_nll_loss.sum() / num_non_empty_sequences
        else:
            return per_batch_nll_loss

### Напишем в общем виде модель для задачи NMT, которая будет поддерживать общий API моделей LSTM и Transformer описанных выше

In [ ]:
class NMTModel(torch.nn.Module):
    """
    Model for NMT task.

    Parameters
    ----------
    vocab : `Vocabulary`, required
        Vocabulary with label encoding and label decoding constructed on datasets.
    embeddings : `torch.nn.ModuleDict`, required
        Torch ModuleDict of embeddings. It should have source and target keys.
    encoder : `torch.nn.Module`, required
        Encoder module for input tokens.
    decoder : `torch.nn.Module`, required
        Decoder module for encoder output.
    """

    def __init__(
        self,
        vocab: Vocabulary,
        embeddings: torch.nn.ModuleDict,
        # Better use seperate Interface for encoder and decoder
        # but for now let's keep it simple stupid and use torch.nn.Module
        encoder: torch.nn.Module,
        decoder: torch.nn.Module,
        device: str,
    ) -> None:
        super().__init__()
        self.vocab = vocab
        self.device = device
        self._embeddings = embeddings
        self._encoder = encoder
        self._decoder = decoder
        self._projection = torch.nn.Linear(
            in_features=self._decoder.get_output_size(),
            out_features=vocab.get_size("target")
        )
        self._loss = SequenceNLLLoss()

    def forward(
        self,
        source_tokens: torch.Tensor,
        target_tokens: torch.Tensor,
        source_mask: torch.Tensor,
        target_mask: torch.Tensor,
    ) -> Dict[str, torch.Tensor]:
        # source_tokens ~ (batch size, sequence length)
        # target_tokens ~ (batch size, sequence length)
        # source_mask ~ (batch size, sequence length)
        # target_mask ~ (batch size, sequence length)
        # 1) Embed source and run encoder
        encoder_output = self._encoder(
            self._embeddings["source"](source_tokens),
            mask=source_mask
        )
        # 2) Embed target and run encoder
        decoder_output = self._decoder(
            self._embeddings["target"](target_tokens),
            encoder_output=encoder_output,
            mask=target_mask
        )
        # 3) Project output to vocabulary
        output = self._projection(decoder_output)
        # 4) Construct output dict
        output_dict = {"logits": output, "probs": torch.softmax(output, dim=-1)}
        output_dict["prediction"] = torch.argmax(output_dict["probs"], dim=-1)
        output_dict["loss"] = self._loss(
            logits=output_dict["logits"],
            target=target_tokens,
            weights=target_mask.float(),
        )
        return output_dict

    def decode(self, indices: List[int], field: str) -> List[str]:
        """Decode sequence of `indices` based on `filed` in Vocabulary."""
        return [self.vocab.index2word[field][int(idx)] for idx in indices]

### Ниже уже знакомый вам train-evaluation loop

In [ ]:
def train_epoch(
    model: torch.nn.Module,
    data_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    return_losses: bool = False,
) -> Union[Dict[str, float], Tuple[Dict[str, float], List[float]]]:
    model.train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for batch in data_loader:
            # Move Batch to GPU
            batch = batch.to_device(model.device, non_blocking=True)
            # Get model results
            output_dict = model(**batch)
            loss = output_dict['loss']
            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            prbar.set_description(f"Loss: {loss.item():.4f}")
            prbar.update(1)
            total_loss += loss.item()
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(
    model: torch.nn.Module,
    data_loader: DataLoader,
) -> Dict[str, float]:
    model.eval()
    total_loss = 0
    num_batches = 0
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for batch in data_loader:
            # Move Batch to GPU
            batch = batch.to_device(model.device, non_blocking=True)
            # Get model results
            output_dict = model(**batch)
            loss = output_dict['loss']
            # Update descirption for tqdm
            prbar.set_description(f"Loss: {loss.item():.4f}")
            prbar.update(1)
            total_loss += loss.item()
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    return metrics


def random_sample_results(
    model: torch.nn.Module,
    batch: CollateBatch,
) -> None:
    """Randomly sample from batch to log model results."""
    batch = batch.to_device(model.device, non_blocking=True)
    result = model(**batch)
    # Get one sample to log
    idx = random.randint(0, len(batch))
    # Print results
    print(
        f"#############\n"
        f"RANDOM SAMPLE:\n"
        f"Source sample: {' '.join(model.decode(batch['source_tokens'][idx], field='source'))}\n"
        f"Target sample: {' '.join(model.decode(batch['target_tokens'][idx], field='target'))}\n"
        f"Predict sample: {' '.join(model.decode(result['prediction'][idx], field='target'))}\n"
        f"#############"
    )

In [ ]:
class LossInfo(NamedTuple):
    full_train_losses: List[float]
    train_epoch_losses: List[float]
    eval_epoch_losses: List[float]


EPOCHS = 10
LR = 0.001

In [ ]:
def fit(
    model: torch.nn.Module,
    epochs: int,
    train_data_loader: DataLoader,
    validation_data_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    show_random: bool = True
) -> LossInfo:
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_epoch(
            model=model,
            data_loader=train_data_loader,
            optimizer=optimizer,
            return_losses=True,
        )
        print(f"Train step loss: {train_metrics['loss']:.4f}")
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics['loss'])
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics = validate(
                model=model,
                data_loader=validation_data_loader,
            )
            print(f"Validation step loss: {validation_metrics['loss']:.4f}")
            if show_random:
                # Get random batch
                batch = next(iter(validation_data_loader))
                random_sample_results(model, batch)
        # Save eval losses
        epoch_eval_losses.append(validation_metrics['loss'])
    return LossInfo(all_train_losses, epoch_train_losses, epoch_eval_losses)

### Ну что же... Перейдём к обучению моделей

#### Сначала LSTM

In [ ]:
# Сначала encoder
input_size = 128
encoder_hidden_size = 256
encoder = LSTMEncoder(
    input_size=input_size,
    hidden_size=encoder_hidden_size,
    # Dropout in LSTM Module is activated only if pass num_layers > 1
    dropout=0.0,
)
# Теперь decoder
decoder = LSTMDecoder(
    input_size=encoder_hidden_size,
    hidden_size=encoder_hidden_size,
    # Dropout in LSTM Module is activated only if pass num_layers > 1
    dropout=0.0,
)
# Ну и объединим это всё в одну модель
lstm_model = NMTModel(
    vocab=vocab,
    embeddings=torch.nn.ModuleDict({
        "source": torch.nn.Embedding(vocab.get_size("source"), input_size, padding_idx=0),
        "target": torch.nn.Embedding(vocab.get_size("target"), encoder_hidden_size, padding_idx=0),
    }),
    encoder=encoder,
    decoder=decoder,
    device=DEVICE,
).to(device=DEVICE)
print(f"Количество параметров: {sum(p.numel() for p in lstm_model.parameters())}")

Количество параметров: 3548224


In [ ]:
loss_info = fit(
    model=lstm_model,
    epochs=EPOCHS,
    train_data_loader=train_dataloader,
    validation_data_loader=validation_dataloader,
    optimizer=torch.optim.Adam(lstm_model.parameters(), lr=LR)
)

Train Epoch: 0
Loss: 0.3896: 100%|██████████| 334/334 [00:39<00:00,  8.47it/s]
Train step loss: 0.4086
Validation Epoch: 0
Loss: 0.1354: 100%|██████████| 84/84 [00:03<00:00, 25.86it/s]
Validation step loss: 0.3299
#############
RANDOM SAMPLE:
Source sample: vous etes vraiment embetants . <end> <pad> <pad> <pad> <pad>
Target sample: <start> you re really annoying . <end> <pad> <pad> <pad> <pad>
Predict sample: <start> you re really annoying . <end> gentleman gentleman gentleman gentleman
#############
Train Epoch: 1
Loss: 0.1267:  39%|███▉      | 130/334 [00:16<00:25,  7.90it/s]


KeyboardInterrupt: 

#### Теперь Transformer

In [ ]:
# Сначала encoder
encoder = TransformerEncoder(
    input_size=128,
    num_layers=1,
    feedforward_hidden_dim=512,
    num_attention_heads=2,
)
# Теперь decoder
decoder = TransformerDecoder(
    input_size=encoder.get_output_size(),
    num_layers=1,
    feedforward_hidden_dim=512,
    num_attention_heads=2,
)
# Ну и объединим это всё в одну модель
transformer_model = NMTModel(
    vocab=vocab,
    embeddings=torch.nn.ModuleDict({
        "source": torch.nn.Embedding(vocab.get_size("source"), 128),
        "target": torch.nn.Embedding(vocab.get_size("target"), 128),
    }),
    encoder=encoder,
    decoder=decoder,
    device=DEVICE,
).to(device=DEVICE)
print(f"Количество параметров: {sum(p.numel() for p in transformer_model.parameters())}")

Количество параметров: 2531392


In [ ]:
loss_info = fit(
    model=transformer_model,
    epochs=EPOCHS,
    train_data_loader=train_dataloader,
    validation_data_loader=validation_dataloader,
    optimizer=torch.optim.Adam(transformer_model.parameters(), lr=LR)
)

Train Epoch: 0
Loss: 0.6817: 100%|██████████| 334/334 [00:27<00:00, 12.00it/s]
Train step loss: 1.1791
Validation Epoch: 0
Loss: 0.1893: 100%|██████████| 84/84 [00:01<00:00, 42.13it/s]
Validation step loss: 0.4489
#############
RANDOM SAMPLE:
Source sample: vous etes vraiment embetants . <end> <pad> <pad> <pad> <pad>
Target sample: <start> you re really annoying . <end> <pad> <pad> <pad> <pad>
Predict sample: <start> you re really annoying . <end> problems problems problems problems
#############
Train Epoch: 1
Loss: 0.3535:  32%|███▏      | 106/334 [00:09<00:19, 11.44it/s]


KeyboardInterrupt: 

## BERT
BERT - это просто несколько stacked Transformer Encoder слоёв из общей модели Transformer + задача Masked Language Modelling.

**Зачем нужен вообще Masked Language Modelling?**

Оригинально BERT претрейнили на задачу Language Modelling, ну и как можно догадаться язык у нас строится последовательно слово за слово, но обычный Transformer так не умеет, так как это bidirectional модель. Такая архитектура не подходит для Language Modelling в общей постановке, так как каждое слово будет косвенно видеть само себя в контекста. Тогда создатели статьи решили применить задачу Cloze или же Masked Language Modelling.

Особенность Masked Language Modelling заключается в том, что мы случайном образом ставим вместо слова **[MASK]** и пытаемся по контексту предсказать тот токен, который там был изначально.

## BPE
Кроме того, в BERT активно используется такая вещь как Byte Pair Encoding. С помощью него решают проблему очень большого вокабуляра для языка, BPE позволяет регулировать его размер. Грубо говоря BPE сегментация оставляет самые частотные токены в датасете, а редкие разделяет на несколько токенов. Это, как можно догадаться, решает проблему дисбаланаса классов, когда наша модель почти не будет предсказывать редкие слова. Притом BPE потенциально позволяет модели научиться распознавать морфологию языка, композицию слов и даже транслитерацию.
* Вот ссылочка на отличную статью на тему BPE: https://arxiv.org/abs/1910.13267
* Для BPE я люблю использовать SentencePiece от Google.

Тут ещё стоит добавить, что некоторые из вас, читая статью от Google про BERT, могли заметить такую вещь как WordPiece, которую использовали исследователи для BPE токенизации. WordPiece отличается от SentencePiece тем, что строит вокабуляр не частотно как последний, а через максимизацию правдоподбия. Теоретически это должно быть эффективно, однако ребята, написавшие RoBERTa, проверили это и пришли к выводу, что частотного подхода достаточно.

## GPT
![SegmentLocal](media/gpt-3-example.gif "GPT-3")